# SasRec training/inference with stream dataset example

## Imports and session initialization

In [6]:
import copy

import torch
import lightning as L
from pyspark.sql import functions as F
from pyspark.sql.window import Window

from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.data import (
    FeatureHint,
    FeatureSource,
    FeatureType,
)
from replay.data.nn import (
    TensorFeatureInfo,
    TensorFeatureSource,
    TensorSchema,
)
from replay.metrics import MAP, OfflineMetrics, Precision, Recall
from replay.splitters import LastNSplitter, RatioSplitter
from replay.utils.session_handler import get_spark_session

# Fix seed to ensure reproducibility
L.seed_everything(42)

import warnings
warnings.filterwarnings("ignore")

Seed set to 42


In [7]:
spark_session = get_spark_session()

25/12/22 12:32:41 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/12/22 12:32:41 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
25/12/22 12:32:41 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


## Preparing data
In this example, we will be using the MovieLens dataset, namely the 100k subset.  
Begin by loading interactions, item features and user features using the created session.

---
**NOTE**

Current implementation of SasRec handles only item and interactions features. It does not take into account user features. As such, they are only used in this example to get complete lists of users.

---

In [8]:
!pip install rs-datasets

In [4]:
from rs_datasets import MovieLens

movielens = MovieLens("100k")

INFO:rs_datasets:Downloading ml-100k from grouplens...
4.94MB [00:02, 1.82MB/s]                            


In [9]:
int_win = Window.partitionBy("user_id").orderBy("item_id")
# interactions = spark_session.createDataFrame(movielens.ratings)
interactions = spark_session.read.option("header", "true").option("sep", "\t").csv("tmp/data/ml-100k/ml-100k.inter")

# NOTE: The following code block is optional and is used
# to counteract the issue of identical timestamps in the dataset.
# Uncomment if you wish to use it.
interactions = (
    interactions.select(["user_id", "item_id", "timestamp"])
    .withColumn("ts", F.col("timestamp").cast("long") * 1000)
    .withColumn("row_num", F.row_number().over(int_win))
    .withColumn("timestamp", (F.col("ts") + F.col("row_num")).cast("string"))
    .drop("ts", "row_num")
)

interactions.show(n=5)

+-------+-------+------------+
|user_id|item_id|   timestamp|
+-------+-------+------------+
|      1|      1|874965758001|
|      1|     10|875693118002|
|      1|    100|878543541003|
|      1|    101|878542845004|
|      1|    102|889751736005|
+-------+-------+------------+
only showing top 5 rows



In [10]:
# user_features = spark_session.createDataFrame(movielens.users)
user_features = spark_session.read.option("header", "true").option("sep", "\t").csv("tmp/data/ml-100k/ml-100k.user")
user_features.show(n=5)

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|      1| 24|     M|technician|   85711|
|      2| 53|     F|     other|   94043|
|      3| 23|     M|    writer|   32067|
|      4| 24|     M|technician|   43537|
|      5| 33|     F|     other|   15213|
+-------+---+------+----------+--------+
only showing top 5 rows



In [11]:
item_features = spark_session.read.option("header", "true").option("sep", "\t").csv("tmp/data/ml-100k/ml-100k.item")
item_features.show(n=5)

+-------+-----------+------------+--------------------+
|item_id|movie_title|release_year|               class|
+-------+-----------+------------+--------------------+
|      1|  Toy Story|        1995|Animation Childre...|
|      2|  GoldenEye|        1995|Action Adventure ...|
|      3| Four Rooms|        1995|            Thriller|
|      4| Get Shorty|        1995| Action Comedy Drama|
|      5|    Copycat|        1995|Crime Drama Thriller|
+-------+-----------+------------+--------------------+
only showing top 5 rows



### Encode catagorical data.
To ensure all categorical data is fit for training, it needs to be encoded using the `LabelEncoder` class. Create an instance of the encoder, providing a `LabelEncodingRule` for each categorcial column in the dataset.

In [12]:
from replay.preprocessing.label_encoder import LabelEncoder, LabelEncodingRule
from replay.utils.types import SparkDataFrame


def encode_data(
    queries: SparkDataFrame, items: SparkDataFrame, interactions: SparkDataFrame, label_encoder: LabelEncoder
):
    full_data = interactions.join(queries, on="user_id").join(items, on="item_id")
    full_data = label_encoder.fit_transform(full_data)

    return full_data

In [13]:
encoder = LabelEncoder(
    [
        LabelEncodingRule("user_id", default_value="last"),
        LabelEncodingRule("item_id", default_value="last"),
        LabelEncodingRule("class", default_value="last"),
    ]
)
encoded_interactions = encode_data(user_features, item_features, interactions, encoder)

25/12/22 12:32:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/22 12:32:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/22 12:32:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/22 12:32:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/22 12:32:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/22 12:32:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/22 1

### Split interactions into the train, validation and test datasets using RatioSplitter

In order to facilitate the model's training, we split the dataset in the following way:
1) A 60/40 data split of original data for training and subsequent splits
2) A 75/25 split of the leftover data for testing/validation respectively (i.e. 30%/10% of the full dataset)

We also remove cold users/items after each split.

In [14]:
train_events, test_events = RatioSplitter(
    test_size=0.4,
    divide_column="user_id",
    query_column="user_id",
    timestamp_column="timestamp",
    drop_cold_users=True,
    drop_cold_items=True,
).split(encoded_interactions)

print(f"{train_events.count()=}, {test_events.count()=}")

train_events.count()=59623, test_events.count()=40171


In [15]:
test_events, val_events = RatioSplitter(
    test_size=0.25,
    divide_column="user_id",
    query_column="user_id",
    timestamp_column="timestamp",
    drop_cold_users=True,
    drop_cold_items=True,
).split(test_events)

print(f"{test_events.count()=}, {val_events.count()=}")

test_events.count()=29782, val_events.count()=10330


### Split the validation dataset into events and ground_truth

For both validation and testing data, the last N items are split into ground truth, which will be used to calculate metrics.

In [16]:
VALIDATION_GROUND_TRUTH_INTERACTIONS_PER_USER = 3
TEST_GROUND_TRUTH_INTERACTIONS_PER_USER = 3

val_events, val_gt = LastNSplitter(
    N=VALIDATION_GROUND_TRUTH_INTERACTIONS_PER_USER,
    divide_column="user_id",
    query_column="user_id",
    strategy="interactions",
).split(val_events)
print(f"{val_events.count()=}, {val_gt.count()=}")

test_events, test_gt = LastNSplitter(
    N=TEST_GROUND_TRUTH_INTERACTIONS_PER_USER, divide_column="user_id", query_column="user_id", strategy="interactions"
).split(test_events)
print(f"{test_events.count()=}, {test_gt.count()=}")

val_events.count()=7535, val_gt.count()=2795
test_events.count()=26953, test_gt.count()=2829


### Dataset preprocessing ("baking")
SasRec expects each user in the batch to provide their events in form of a sequence. For this reason, the event splits must be properly processed using the `groupby_sequences` function provided by RePlay.

In [17]:
from replay.data.nn.utils import groupby_sequences


def bake_data(full_data: SparkDataFrame):
    grouped_interactions = groupby_sequences(events=full_data, groupby_col="user_id", sort_col="timestamp")

    return grouped_interactions

In [18]:
train_events = bake_data(train_events)
val_events = bake_data(val_events)
val_gt = bake_data(val_gt)
test_events = bake_data(test_events)

To ensure we don't validate on unknown users, we join train and validation data by user ids, leaving only the common ones.  
We also pre-package the validation data with its ground truth and train-time events.

In [19]:
# Keep common query ids between val_dataset and val_gt
val_events = val_events.join(val_gt, on="user_id", how="left_semi")
val_gt = val_gt.join(val_events, on="user_id", how="left_semi")

gt_to_join = val_gt.select(["user_id", "item_id"]).withColumnRenamed("item_id", "ground_truth")
train_to_join = train_events.select(["user_id", "item_id"]).withColumnRenamed("item_id", "train")

val_events = val_events.join(gt_to_join, on="user_id", how="left")
val_events = val_events.join(train_to_join, on="user_id", how="left")

TRAIN_LEN = val_events.select(F.max(F.size("train")).alias("res")).collect()[0].res
GT_LEN = val_events.select(F.max(F.size("ground_truth")).alias("res")).collect()[0].res

In [20]:
TRAIN_PATH = "temp/data/train.parquet"
VAL_PATH = "temp/data/val.parquet"
TEST_PATH = "temp/data/test.parquet"

train_events.write.mode("overwrite").parquet(TRAIN_PATH)
val_events.write.mode("overwrite").parquet(VAL_PATH)
test_events.write.mode("overwrite").parquet(TEST_PATH)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model. It is required by the SasRec model to correctly create embeddings at train time.

In [21]:
EMBEDDING_DIM = 64

ITEM_FEATURE_NAME = "item_id"
NUM_UNIQUE_ITEMS = len(encoder.mapping["item_id"])
NUM_UNIQUE_CLASS_VALUES = len(encoder.mapping["class"])

tensor_schema = TensorSchema(
    [
        TensorFeatureInfo(
            name="item_id",
            is_seq=True,
            padding_value=NUM_UNIQUE_ITEMS,
            cardinality=NUM_UNIQUE_ITEMS + 1,  # taking into account padding
            embedding_dim=EMBEDDING_DIM,
            feature_type=FeatureType.CATEGORICAL,
            feature_sources=[TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_id")],
            feature_hint=FeatureHint.ITEM_ID,
        ),
        TensorFeatureInfo(
            name="class",
            is_seq=True,
            padding_value=NUM_UNIQUE_CLASS_VALUES,
            cardinality=NUM_UNIQUE_CLASS_VALUES + 1,  # taking into account padding
            embedding_dim=EMBEDDING_DIM,
            feature_type=FeatureType.CATEGORICAL,
            feature_sources=[TensorFeatureSource(FeatureSource.ITEM_FEATURES, "item_id")],
        ),
    ]
)

### Configure ParquetModule and transformation pipelines
The `ParquetModule` class enables training of models on large datasets by reading data in streaming mode. This class initialized with a metadata dict containing information about dataset's features and miscellanious options for initialization (such as shuffling).

Additionally, `ParquetModule` supports "transform pipelines" - stage-specific modules implementing additional preprocessing to be performed on batch level right before the forward pass.  

In our case, we create the following pipelines:
1) Training:
    1. Create a target, which contains the shifted item sequence that represents the next item in the sequence (for the next item prediction task).
    2. Optionally sample negatives (required only for sampled losses).
    3. Rename features to match it with expected format by the model during training.
    4. Unsquueeze target (`positive_labels`) and it's padding mask (`target_padding_mask`).
    5. Group input features to be embed in expected format.

2) Validation/Inference:
    1. Rename/group features to match it with expected format by the model during valdiation/inference.

Then, metadata for ParquetModule should be created. It contains shape and padding value for each feature.

In [22]:
from replay.nn.transforms import (
    UnsqueezeTransform,
    GroupTransform,
    RenameTransform,
    NextTokenTransform,
    UniformNegativeSamplingTransform,
)

MAX_SEQ_LEN = 50
BATCH_SIZE = 32
SHIFT = 1

TRANSFORMS = {
    "train": [
        NextTokenTransform(
            label_field="item_id", query_features="user_id", shift=SHIFT, out_feature_name="positive_labels"
        ),
        UniformNegativeSamplingTransform(vocab_size=NUM_UNIQUE_ITEMS, num_negative_samples=200),
        RenameTransform(
            {"user_id": "query_id", "item_id_mask": "padding_mask", "positive_labels_mask": "target_padding_mask"}
        ),
        UnsqueezeTransform("target_padding_mask", -1),
        UnsqueezeTransform("positive_labels", -1),
        GroupTransform({"feature_tensors": ["item_id", "class"]}),
    ],
    "val": [
        RenameTransform({"user_id": "query_id", "item_id_mask": "padding_mask"}),
        GroupTransform({"feature_tensors": ["item_id", "class"]}),
    ],
    "test": [
        RenameTransform({"user_id": "query_id", "item_id_mask": "padding_mask"}),
        GroupTransform({"feature_tensors": ["item_id", "class"]}),
    ],
}

shared_meta = {
    "user_id": {},
    "item_id": {"shape": MAX_SEQ_LEN, "padding": tensor_schema["item_id"].padding_value},
    "class": {"shape": MAX_SEQ_LEN, "padding": tensor_schema["class"].padding_value},
}

METADATA = {
    "train": copy.deepcopy(shared_meta),
    "val": {
        **copy.deepcopy(shared_meta),
        "train": {
            "shape": TRAIN_LEN, "padding": tensor_schema["item_id"].padding_value,
        },
        "ground_truth": {
            "shape": GT_LEN, "padding": tensor_schema["item_id"].padding_value,
        },
    },
    "test": copy.deepcopy(shared_meta),
}

In [23]:
from replay.data.nn import ParquetModule

streaming_dataset = ParquetModule(
    train_path=TRAIN_PATH,
    val_path=VAL_PATH,
    test_path=TEST_PATH,
    batch_size=BATCH_SIZE,
    metadata=METADATA,
    transforms=TRANSFORMS,
)

# NOTE: You can also create a module specifically for training/inference by providing only their respective datapaths
# streaming_dataset_train_only = ParquetModule(
#     train_path=TRAIN_PATH,
#     val_path=VAL_PATH,
#     batch_size=BATCH_SIZE,
#     metadata=METADATA,
#     transforms=TRANSFORMS
# )

## Train model
### Create SasRec model instance and run the training stage using lightning
We may now train the model using the Lightning trainer class. 

RePlay's implementation of SasRec is designed in a modular, **block-based approach**. Instead of passing configuration parameters to the constructor, SasRec is now built by providing fully initialized components that makes the model more flexible and easier to extend. SasRec consists of the following components: embedder, aggregator, encoder, mask, output_normalization, loss.

#### Components

* `Embedder` -The embedder is responsible for converting input features into embeddings. The default implementation is `SequenceEmbedding`, which supports the following feature types: categorical, categorical_list, numerical, numerical_list

* `Aggregator` - The aggregator combines all embeddings produced by the embedder and adds positional embeddings.
Currently, `SasRecAggregator` is supported. It internally uses one of the following embedding aggregation strategies: `SumAggregator`, `ConcatAggregator`.

* `Encoder` - The encoder represents the core transformer block of the model. The following implementations are currently available: `SasRecTransformerLayer` (default one), `DiffAttentionLayer` (a modified version with differential attention).

* `Mask` - The mask is an object that creates attention mask by input. RePlay supports `DefaultAttentionMask` creating a lower-triangular attention mask.

* `Output Normalization` - Any suitable PyTorch normalization layer may be used as output_normalization, for example: torch.nn.LayerNorm or torch.nn.RMSNorm

* `Loss` - The loss component defines how the training loss is computed. All available loss implementations are located in nn/loss.


In [24]:
from replay.nn import DefaultAttentionMask, SequenceEmbedding, SumAggregator
from replay.nn.loss import CESampled
from replay.nn.sequential import SasRec, SasRecAggregator, SasRecTransformerLayer


NUM_BLOCKS = 1
NUM_HEADS = 1
DROPOUT = 0.0

sasrec = SasRec(
    embedder=SequenceEmbedding(
        schema=tensor_schema,
        categorical_list_feature_aggregation_method="sum",
    ),
    embedding_aggregator=SasRecAggregator(
        embedding_aggregator=SumAggregator(embedding_dim=EMBEDDING_DIM),
        max_sequence_length=MAX_SEQ_LEN,
        dropout=DROPOUT,
    ),
    attn_mask_builder=DefaultAttentionMask(
        reference_feature_name=tensor_schema.item_id_feature_name,
        num_heads=NUM_HEADS,
    ),
    encoder=SasRecTransformerLayer(
        embedding_dim=EMBEDDING_DIM,
        num_heads=NUM_HEADS,
        num_blocks=NUM_BLOCKS,
        dropout=DROPOUT,
        activation="relu",
    ),
    output_normalization=torch.nn.LayerNorm(EMBEDDING_DIM),
    loss=CESampled(padding_idx=tensor_schema.item_id_features.item().padding_value),
)

#### Default Configuration

Default SasRec model may be created quickly via method build_original. Such model has CE loss, original SasRec transformer layes, and embeddings are aggregated via sum.

In [25]:
default_sasrec = SasRec.build_default(
    schema=tensor_schema, 
    embedding_dim=EMBEDDING_DIM, 
    max_sequence_length=MAX_SEQ_LEN
    )

A universal PyTorch Lightning module is provided that can work with any RePlay NN model.

In [26]:
from replay.nn.lightning import LightningModule
from replay.models.nn.optimizer_utils import FatOptimizerFactory, FatLRSchedulerFactory

model = LightningModule(
    sasrec,
    optimizer_factory=FatOptimizerFactory(),
    lr_scheduler_factory=FatLRSchedulerFactory(),
)

To facilitate training, we add the following callbacks:
1) `ModelCheckpoint` - to save the best trained model based on its Recall metric. It's a default Lightning Callback.
1) `ComputeMetricsCallback` - to display a detailed validation metric matrix after each epoch. It's a custom RePlay callback for computing recsys metrics on validation. It supports model's logits postpocessing (before metrics computing), we will use RePlay `SeenItemsFilter` in order to compute metrics on unseen ground truth items only.


In [28]:
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger
from replay.nn.lightning.callbacks import ComputeMetricsCallback
from replay.nn.lightning.postprocessors import SeenItemsFilter

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    monitor="recall@10",
    mode="max",
)

postprocessors = [
    SeenItemsFilter(
        seen_path=VAL_PATH,
        item_count=NUM_UNIQUE_ITEMS,
        query_column="user_id",
        item_column=tensor_schema.item_id_feature_name,
    )
]
validation_metrics_callback = ComputeMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=NUM_UNIQUE_ITEMS,
    postprocessors=postprocessors
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SasRec-example")

trainer = L.Trainer(
    max_epochs=5,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

trainer.fit(model, datamodule=streaming_dataset)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | SasRec | 150 K  | train
-----------------------------------------
150 K     Trainable params
0         Non-trainable params
150 K     Total params
0.601     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 0, global step 30: 'recall@10' reached 0.08229 (best 0.08229), saving model to '/home/evtsinovnik/replay_master/examples/.checkpoints/epoch=0-step=30.ckpt' as top 1


...

Epoch 3, global step 120: 'recall@10' was not in top 1


k              1        10        20         5
map     0.025000  0.027705  0.033938  0.021319
ndcg    0.025000  0.053176  0.075817  0.035690
recall  0.008333  0.080208  0.143750  0.041667



Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 4, global step 150: 'recall@10' was not in top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


k              1        10        20         5
map     0.034375  0.030553  0.036934  0.026788
ndcg    0.034375  0.055916  0.078547  0.045150
recall  0.011458  0.079167  0.142708  0.055208



We can now laod the best model using the path stored in the callback.

In [29]:
best_model = LightningModule.load_from_checkpoint(checkpoint_callback.best_model_path, model=sasrec)

## Inference stage

### Run inference
We can now perform inference using the data module we created earlier. Recommendations can be fetched in four formats: PySpark DataFrame, Pandas DataFrame, Polars DataFrame or raw PyTorch tensors. Each of the types corresponds a callback. Inthis example, we'll be using the `PandasTopItemsCallback`.
Prediction callbacks also can filter results using postprocessors.

In [48]:

from replay.nn.lightning.callbacks import PandasTopItemsCallback

csv_logger = CSVLogger(save_dir=".logs/test", name="SasRec-example")

TOPK = [1, 5, 10, 20]

postprocessors = [
    SeenItemsFilter(
        seen_path=TEST_PATH,
        item_count=NUM_UNIQUE_ITEMS,
        query_column="user_id",
        item_column=tensor_schema.item_id_feature_name,
    )
]

pandas_prediction_callback = PandasTopItemsCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

trainer = L.Trainer(callbacks=[pandas_prediction_callback], logger=csv_logger, inference_mode=True)

trainer.predict(best_model, datamodule=streaming_dataset, return_predictions=False)

pandas_res = pandas_prediction_callback.get_result()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [49]:
pandas_res

,user_id,item_id,score
0,3,1506,4.373491
0,3,1011,4.034044
0,3,853,4.02631
0,3,1491,3.801472
0,3,1490,3.697806
...,...,...,...
942,938,1052,2.429619
942,938,1374,2.427046
942,938,679,2.367162
942,938,1649,2.346843


### Calculating metrics

In [50]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK)], query_column="user_id", rating_column="score"
)(pandas_res, test_gt.toPandas())

In [51]:
metrics_to_df(result_metrics)

k,1,10,20,5
MAP,0.053022,0.045232,0.053110,0.037127
Precision,0.053022,0.034783,0.028897,0.041145
Recall,0.017674,0.115942,0.192648,0.068575
